In [1]:
import torch
import time
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()

    print('There are %d GPU(s) available.' % n_gpu)

    print('We will use the GPU:', [torch.cuda.get_device_name(i) for i in range(n_gpu)])

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [134]:
import pandas as pd
import numpy as np
df = pd.read_csv("./lda-title-study2-bert.csv")
df["trend"] = df["trend"].replace("A",1)
df["trend"] = df["trend"].replace("B",2)
df["trend"] = df["trend"].replace("C",3)

# df = df.drop('fraud_y',axis = 1)
time1 = time.time()
df.head(5)

,date,title,price,next_price,trend,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,...,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20,Topic 21
0,2021/10/15 22:00,THE AMERICAN DREAM IS DEAD AND BITCOIN HAS REP...,61209.42,61396.22,3,0.007305,0.068333,0.071524,0.012446,0.228208,...,0.008312,0.018644,0.001095,0.007500,0.005390,0.012560,0.007618,0.002308,0.001313,0.009082
1,2021/10/15 20:00,BITCOIN EDUCATION FOR INDONESIA,61446.33,61372.13,1,0.010861,0.101592,0.025216,0.099581,0.014846,...,0.012358,0.027718,0.001628,0.011151,0.008014,0.018673,0.011326,0.003431,0.001952,0.013502
2,2021/10/15 18:00,QUANTIFYING THE BITCOIN SUPPLY SHORTAGE,61762.01,61644.99,2,0.010861,0.101588,0.025215,0.018503,0.014845,...,0.012357,0.027717,0.001628,0.011150,0.008013,0.018672,0.011325,0.003431,0.001952,0.094614
3,2021/10/15 15:38,SENATOR CYNTHIA LUMMIS ON US DEBT LIMIT RAISE:...,59792.01,59885.73,2,0.006927,0.064797,0.016083,0.011802,0.009469,...,0.007882,0.379837,0.001038,0.007112,0.005111,0.063650,0.007224,0.002188,0.001245,0.008612
4,2021/10/15 13:19,MASSMUTUAL BITCOIN INVESTMENT HAS TRIPLED IN D...,59789.10,59781.47,2,0.008200,0.076702,0.202766,0.013970,0.072451,...,0.009330,0.020927,0.001229,0.008419,0.006050,0.014098,0.008551,0.002591,0.001474,0.010194


In [135]:
def get_split(text1):
    l_total = []
    l_partial = []
    if len(text1.split())//150>0:
        n = len(text1.split())//150
    else:
        n = 1
    for w in range(n):
        if w == 0:
            l_partial = text1.split()[:200]
            l_total.append(" ".join(l_partial))
        else:
            l_partial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_partial))
    return l_total

In [136]:
from datetime import date
from posixpath import split


df['title_split'] = df['title'].apply(get_split)
# df = df.drop(columns=['date,'price','next_price','Topic 0' ,'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7', 'Topic 8', 'Topic 9', 'Topic 10','Topic 11','Topic 12', 'Topic 13', 'Topic 14','Topic 15','Topic 16','Topic 17','Topic 18','Topic 19','Topic 20','Topic 21' ] )
# df=df.drop(columns= ['date'], index_col = 0)
df.head()

,date,title,price,next_price,trend,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,...,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20,Topic 21,title_split
0,2021/10/15 22:00,THE AMERICAN DREAM IS DEAD AND BITCOIN HAS REP...,61209.42,61396.22,3,0.007305,0.068333,0.071524,0.012446,0.228208,...,0.018644,0.001095,0.007500,0.005390,0.012560,0.007618,0.002308,0.001313,0.009082,[THE AMERICAN DREAM IS DEAD AND BITCOIN HAS RE...
1,2021/10/15 20:00,BITCOIN EDUCATION FOR INDONESIA,61446.33,61372.13,1,0.010861,0.101592,0.025216,0.099581,0.014846,...,0.027718,0.001628,0.011151,0.008014,0.018673,0.011326,0.003431,0.001952,0.013502,[BITCOIN EDUCATION FOR INDONESIA]
2,2021/10/15 18:00,QUANTIFYING THE BITCOIN SUPPLY SHORTAGE,61762.01,61644.99,2,0.010861,0.101588,0.025215,0.018503,0.014845,...,0.027717,0.001628,0.011150,0.008013,0.018672,0.011325,0.003431,0.001952,0.094614,[QUANTIFYING THE BITCOIN SUPPLY SHORTAGE]
3,2021/10/15 15:38,SENATOR CYNTHIA LUMMIS ON US DEBT LIMIT RAISE:...,59792.01,59885.73,2,0.006927,0.064797,0.016083,0.011802,0.009469,...,0.379837,0.001038,0.007112,0.005111,0.063650,0.007224,0.002188,0.001245,0.008612,[SENATOR CYNTHIA LUMMIS ON US DEBT LIMIT RAISE...
4,2021/10/15 13:19,MASSMUTUAL BITCOIN INVESTMENT HAS TRIPLED IN D...,59789.10,59781.47,2,0.008200,0.076702,0.202766,0.013970,0.072451,...,0.020927,0.001229,0.008419,0.006050,0.014098,0.008551,0.002591,0.001474,0.010194,[MASSMUTUAL BITCOIN INVESTMENT HAS TRIPLED IN ...


In [137]:
df_l = []
label_l = []
index_l =[]
for idx,row in df.iterrows():
  for l in row['title_split']:
    df_l.append(l)
    label_l.append(row['trend'])
    index_l.append(idx)
len(df_l), len(label_l), len(index_l)

(1280, 1280, 1280)

In [177]:
train_df = pd.DataFrame({'title':df_l, 'trend':label_l})
train_df.head()


,title,trend
0,THE AMERICAN DREAM IS DEAD AND BITCOIN HAS REP...,3
1,BITCOIN EDUCATION FOR INDONESIA,1
2,QUANTIFYING THE BITCOIN SUPPLY SHORTAGE,2
3,SENATOR CYNTHIA LUMMIS ON US DEBT LIMIT RAISE:...,2
4,MASSMUTUAL BITCOIN INVESTMENT HAS TRIPLED IN D...,2


In [178]:
cates = train_df.groupby('trend')
print("total categories:", cates.ngroups)
print(cates.size())

total categories: 3
trend
1    466
2    422
3    392
dtype: int64


In [179]:

import torch
from transformers import BertTokenizer
from IPython.display import clear_output

Pretrain_model_name = "bert-base-uncased"  # 英文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(Pretrain_model_name)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.11.0


In [180]:
sentences = train_df['title'].values
labels = train_df["trend"] 

In [211]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # 輸入文字
                        add_special_tokens = True, # 新增 '[CLS]' 和 '[SEP]'
                        max_length = 5,           # 填充 & 截斷長度
                        pad_to_max_length = True,
                        return_attention_mask = True,   # 返回 attn. masks.
                        return_tensors = 'pt',     # 返回 pytorch tensors 格式的資料
                   )
    
    # 將編碼後的文字加入到列表  
    input_ids.append(encoded_dict['input_ids'])
    
    # 將文字的 attention mask 也加入到 attention_masks 列表
    attention_masks.append(encoded_dict['attention_mask'])

# 將列表轉換為 tensor
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# 輸出第 1 行文字的原始和編碼後的資訊
#print('Original: ', sentences[0])
print('Token IDs:', input_ids.shape)
print('attention_masks IDs:', attention_masks.shape)
print('labels IDs:', labels.shape)

Token IDs: torch.Size([1280, 5])
attention_masks IDs: torch.Size([1280, 5])
labels IDs: torch.Size([1280])


/Users/danielpeng/Programs/others/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-211-a75037f21891>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [212]:
from torch.utils.data import TensorDataset, random_split
dataset = TensorDataset(input_ids, attention_masks, labels)
# 計算訓練集和驗證集大小
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
# 按照資料大小隨機拆分訓練集和測試集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_size = int(0.9 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} testing samples'.format(test_size))
print('{:>5,} validation samples'.format(val_size))

  806 training samples
   90 testing samples
  384 validation samples


In [213]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 在 fine-tune 的訓練中，BERT 作者建議小批量大小設為 16 或 32
batch_size = 32
# 為訓練和驗證集建立 Dataloader，對訓練樣本隨機洗牌

train_dataloader = DataLoader(train_dataset,sampler = RandomSampler(train_dataset), batch_size = batch_size)
    

validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size)

In [214]:
from transformers import BertForSequenceClassification

#BertForSequenceClassification是經過預訓練的BERT模型，最後具有單個線性分類層。
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [216]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [217]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
from torch import optim
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
model.cpu()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [218]:
def train(input_data, model, optimizer):
    model.train()

    loss_list = []
    total_count = 0
    acc_count = 0
    for data in input_data:
        text, masks_tensors, labels = data[0].to(device), data[1].to(device),data[2].to(device).long()
        optimizer.zero_grad()
        train_pred = model(text,attention_mask = masks_tensors, labels=labels)
        
        #print(train_pred)
        batch_loss = train_pred[0]
        logits = train_pred[1]
        batch_loss.backward()
        optimizer.step()
        
        #_, pred = torch.max(batch_loss.data, 1)
        total_count += text.size(0)
        acc_count += np.sum(np.argmax(logits.cpu().data.numpy(), axis=1)== data[2].numpy())
        loss_list.append(batch_loss.item())
        #print(batch_loss.item())
        ########################################################################
        #                           End of your code                           #
        ########################################################################
    # Compute this epoch accuracy and loss
    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

In [219]:
from sklearn.metrics import classification_report
def val(input_data, model):
    model.eval()

    loss_list = []
    acc_list = []
    data_list = []
    total_count = 0
    acc_count = 0

    with torch.no_grad():
        for data in input_data:
            text,masks_tensors,labels = data[0].to(device), data[1].to(device), data[2].to(device).long()

            val_pred = model(text,attention_mask=masks_tensors, labels=labels)
            
            batch_val_loss = val_pred[0]
            logits = val_pred[1]
            
            total_count += text.size(0)
            acc_count += np.sum(np.argmax(logits.cpu().data.numpy(), axis=1) == data[2].numpy())
            acc_list.extend(np.argmax(logits.cpu().data.numpy(), axis=1))
            data_list.extend(data[2].numpy())
            loss_list.append(batch_val_loss.item())
    fper,tper,threshold = roc_curve(acc_list, data_list)
    roc_auc = metrics.auc(fper, tper)
    print(roc_auc)
    print(cohen_kappa_score(acc_list, data_list))
    print(classification_report(acc_list, data_list))        
    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss 

In [220]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
################################################################################
# You can adjust those hyper parameters to loop for max_epochs times           #
################################################################################
max_epochs = 10
log_interval = 1 # print acc and loss in per log_interval time
################################################################################
#                               End of your code                               #
################################################################################
train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []


for epoch in range(1, max_epochs + 1):
    train_acc, train_loss = train(train_dataloader, model, optimizer)
    val_acc, val_loss = val(validation_dataloader, model)

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)
    
    
    if epoch % log_interval == 0:
        print('=' * 20, 'Epoch', epoch, '=' * 20)
        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss))
        print('Val Acc: {:.6f}   Val Loss: {:.6f} '.format(val_acc, val_loss))

IndexError: Target 3 is out of bounds.

In [ ]:
import matplotlib.pyplot as plt
acc = train_acc_list
val_acc = val_acc_list
loss = train_loss_list
val_loss = val_loss_list
epochs = range(1,11)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import classification_report
def test(input_data, model):
    model.eval()
    loss_list = []
    acc_list = []
    data_list = []
    total_count = 0
    acc_count = 0

    with torch.no_grad():
        for data in input_data:
            text,masks_tensors,labels = data[0].to(device), data[1].to(device), data[2].to(device).long()

            test_pred = model(text,attention_mask=masks_tensors, labels=labels)
            
            batch_val_loss = test_pred[0]
            logits = test_pred[1]
            
            total_count += text.size(0)
            acc_count += np.sum(np.argmax(logits.cpu().data.numpy(), axis=1) == data[2].numpy())
            acc_list.extend(np.argmax(logits.cpu().data.numpy(), axis=1))
            data_list.extend(data[2].numpy())
            loss_list.append(batch_val_loss.item())
    fper,tper,threshold = roc_curve(acc_list, data_list)
    roc_auc = metrics.auc(fper, tper)
    print(roc_auc)
    plot_roc_curve(fper, tper)
    print("AUC value:",metrics.auc(fper, tper))
    print(cohen_kappa_score(acc_list, data_list))
    print(classification_report(acc_list, data_list))        
    acc = acc_count / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss 

In [ ]:
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [ ]:
test_acc_list = []
test_loss_list = []
test_acc, test_loss = test(test_dataloader, model)
print('test Acc: {:.6f} test Loss: {:.6f}'.format(test_acc, test_loss))